<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to check if OBS have been assigned to the correct layer.
</span><br>

## 0.0. Libraries

In [1]:
import WS_Mdl as WS # import functions defined for WS_Mdl (by me)
import osfrom os import listdir as LD, makedirs as MDsfrom os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE 
import imod
import pandas as pd
from pathlib import Path
import re
import numpy as np
import geopandas as gpd
import xarray as xr

## 0.1. Options

In [2]:
MdlN = 'NBr5'
Mdl = ''.join([c for c in MdlN if not c.isdigit()])
d_Pa = WS.get_MdlN_paths(MdlN)

# 1. Read INI file to get model size

In [3]:
Xmin, Ymin, Xmax, Ymax, cellsize, N_R, N_C = WS.Mdl_Dmns_from_INI(d_Pa['Pa_INI'])

# 2. Read IPF

In [4]:
Pa_IPF = PJ(d_Pa['Pa_Mdl'], r'In\OBS\NBr5\ijkset_selectie\ijkset_selectie.ipf')
IPF = WS.read_IPF_Spa(Pa_IPF)

# 3. Process IPF

In [5]:
IPF.columns = IPF.columns.str.replace(',-9999','') # Headers had ',-9999' in their names. They serve no purpose, hence will be removed.

In [6]:
IPF.rename(columns={'xcoordinate': 'x', 'ycoordinate': 'y'}, inplace=True)

In [7]:
IPF_Mdl = IPF.loc[ ( (IPF['x'] > Xmin) & (IPF['x'] < Xmax) ) & ( (IPF['y'] > Ymin) & (IPF['y'] < Ymax) ) ].copy() # Limit to OBS within the Mdl Aa

The MF6 layer needs to be calculated, as ilay represents the old model layer.

In [8]:
IPF_Mdl['L'] = IPF_Mdl['ilay']*2 - 1

In [9]:
IPF_Mdl['R'] = ( -(IPF_Mdl['y']-Ymax) / cellsize ).astype(np.int32) + 1 # Ymax is at the origin of the model.
IPF_Mdl['C'] = (  (IPF_Mdl['x']-Xmin) / cellsize ).astype(np.int32) + 1 # Xmin at the origin of the model.

In [10]:
IPF_Mdl.sort_values(by=["L", "R", "C"], ascending=[True, True, True], inplace=True) # Let's sort the DF by L, R, C

# 4. Read TOP and BOT arrays 

In [11]:
Pa_TOP = r'C:\OD\WS_Mdl\models\NBr\In\TOP'
l_TOP = [PJ(Pa_TOP, i) for i in LD(Pa_TOP) if 'idf' in i]
TOP = imod.formats.idf.open(l_TOP, pattern="{name}_L{layer}_").sel(x=slice(Xmin, Xmax), y=slice(Ymax, Ymin))

In [12]:
Pa_BOT = r'C:\OD\WS_Mdl\models\NBr\In\BOT'
l_BOT = [PJ(Pa_BOT, i) for i in LD(Pa_BOT) if 'idf' in i]
BOT = imod.formats.idf.open(l_BOT, pattern="{name}_L{layer}_").sel(x=slice(Xmin, Xmax), y=slice(Ymax, Ymin))

# 5. Match L based on TOP and BOT values (compared to filter top and bot)

In [13]:
# Make sure x and y coordinates exist in TOP as 1D arrays
TOP_x = TOP.coords['x'].values
TOP_y = TOP.coords['y'].values

In [14]:
# Prepare output containers
case1_layers, case2_layers, case3_layers, case4_layers, combined_layers, match_distances, combined_TOP_vals, combined_BOT_vals = ([] for i in range(8))

In [15]:
for _, row in IPF_Mdl.iterrows(): # Loop through each row in IPF_Mdl
    # Find closest grid point in TOP
    dx = TOP_x - row['x']
    dy = TOP_y - row['y']
    dist_matrix = np.sqrt((dy[:, None])**2 + (dx[None, :])**2)
    iy, ix = np.unravel_index(np.argmin(dist_matrix), dist_matrix.shape)
    min_dist = dist_matrix[iy, ix]

    flt_top = row['filtertoplevel']
    flt_bot = row['filterbottomlevel']
    n_layers = TOP.shape[0]

    # Initialize per-row results
    layer_case1 = None
    layer_case2 = None
    layer_case3 = None
    layers_case4 = []

    # Loop over layers
    for k in range(n_layers):
        top_k = float(TOP[k, iy, ix].values)
        bot_k = float(BOT[k, iy, ix].values)

        in_top  = bot_k < flt_top < top_k
        in_bot  = bot_k < flt_bot < top_k
        in_full = flt_bot < bot_k and flt_top > top_k

        if in_top:
            layer_case1 = k
        if in_bot:
            layer_case2 = k
        if in_top and in_bot:
            layer_case3 = k
        if in_full:
            layers_case4.append(k)

    # Combine and convert to 1-based indexing
    matched_layers = sorted(set(
        [l for l in [layer_case1, layer_case2, layer_case3] if l is not None] + layers_case4
    ))
    matched_layers_1based = [l + 1 for l in matched_layers]

    # Get combined TOP and BOT if any layer matched
    if matched_layers:
        shallowest_k = min(matched_layers)
        deepest_k    = max(matched_layers)
        top_val = float(TOP[shallowest_k, iy, ix].values)
        bot_val = float(BOT[deepest_k,    iy, ix].values)
    else:
        top_val = None
        bot_val = None

    # Store results (with 1-based indexing)
    case1_layers.append(layer_case1 + 1 if layer_case1 is not None else None)
    case2_layers.append(layer_case2 + 1 if layer_case2 is not None else None)
    case3_layers.append(layer_case3 + 1 if layer_case3 is not None else None)
    case4_layers.append([l + 1 for l in layers_case4])
    combined_layers.append(matched_layers_1based)
    combined_TOP_vals.append(top_val)
    combined_BOT_vals.append(bot_val)
    match_distances.append(min_dist)

In [16]:
IPF_Mdl_ = IPF_Mdl.copy() # copy, in case it doesn't workout or we want to test some things.

In [17]:
# Assign to DataFrame
IPF_Mdl_['case1_L']     = case1_layers
IPF_Mdl_['case2_L']     = case2_layers
IPF_Mdl_['case3_L']     = case3_layers
IPF_Mdl_['case4_L']     = case4_layers
IPF_Mdl_['L_match']  = combined_layers
IPF_Mdl_['TOP_L_match']    = combined_TOP_vals
IPF_Mdl_['BOT_L_match']    = combined_BOT_vals
IPF_Mdl_['match_distance']  = match_distances


In [18]:
IPF_Mdl_.sort_values('R', inplace=True)

In [19]:
IPF_Mdl_

,x,y,ilay,id,code,filterno,surfacelevel,filtertoplevel,filterbottomlevel,L,R,C,case1_L,case2_L,case3_L,case4_L,L_match,TOP_L_match,BOT_L_match,match_distance
133,113632.0,396195.0,2,5240_1,5240,1,0.89,0.89,-0.11,3,1,22,1,1,1.0,[],[1],1.830000,-0.154000,48.466483
187,113322.0,396099.0,3,5357_2,5357,2,2.31,1.05,0.06,5,5,9,5,5,5.0,[],[5],1.610000,-0.780000,56.435804
4059,124540.0,396095.0,4,B50E0379_1,B50E0379,1,-9999.00,8.54,6.54,7,5,458,5,7,NaN,[6],"[5, 6, 7]",12.830000,-2.917400,46.097722
145,115057.0,396050.0,3,5256_1,5256,1,2.44,2.44,1.44,5,7,79,5,5,5.0,[],[5],3.190000,0.780000,7.000000
3934,115060.0,396050.0,3,B50B0555_2,B50B0555,2,3.50,1.72,1.42,5,7,79,5,5,5.0,[],[5],3.190000,0.780000,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3862,118530.0,388835.0,10,B50B0056_1,B50B0056,1,14.77,-35.99,-37.49,19,295,218,19,19,19.0,[],[19],-13.110000,-39.029999,25.000000
3896,116015.0,388490.0,12,B50B0210_2,B50B0210,2,11.09,-40.91,-42.91,23,309,117,23,23,23.0,[],[23],-35.009998,-51.970001,53.150729
3897,116015.0,388490.0,18,B50B0210_3,B50B0210,3,11.09,-82.91,-84.91,35,309,117,35,35,35.0,[],[35],-71.089996,-93.449997,53.150729
3895,116015.0,388490.0,9,B50B0210_1,B50B0210,1,11.09,-0.91,-2.91,17,309,117,17,17,17.0,[],[17],0.010000,-7.100000,53.150729


In [20]:
def remove_L_from_combined_str(row):
    try:
        layer_list = row['L_match'].copy()
        layer_list.remove(row['L'])
    except ValueError:
        layer_list = row['L_match']
    return ', '.join(str(x) for x in layer_list)

In [21]:
# Checks
IPF_Mdl_['match?'] = IPF_Mdl_.apply(lambda row: row['L'] in row['L_match'], axis=1)
IPF_Mdl_['L_extra'] = IPF_Mdl_.apply(remove_L_from_combined_str, axis=1)
IPF_Mdl_['TOP_match?'] = IPF_Mdl_.apply(lambda R: True if R['TOP_L_match'] > R['filtertoplevel'] else False, axis=1)
IPF_Mdl_['BOT_match?'] = IPF_Mdl_.apply(lambda R: True if R['BOT_L_match'] < R['filterbottomlevel'] else False, axis=1)

In [22]:
IPF_Mdl_['case4_L'] = IPF_Mdl_['case4_L'].apply(lambda x: ', '.join(str(i) for i in x))
IPF_Mdl_['L_match'] = IPF_Mdl_['L_match'].apply(lambda x: ', '.join(str(i) for i in x))

In [31]:
IPF_Mdl_[['case4_L', 'L_match', 'L_extra']] = IPF_Mdl_[['case4_L', 'L_match', 'L_extra']].astype(str)

In [33]:
IPF_Mdl_.sort_values('id')

,x,y,ilay,id,code,filterno,surfacelevel,filtertoplevel,filterbottomlevel,L,...,case3_L,case4_L,L_match,TOP_L_match,BOT_L_match,match_distance,match?,L_extra,TOP_match?,BOT_match?
86,114730.0,390157.0,9,5028_2,5028,2,0.00,-0.98,-1.98,17,...,17.0,,17,-0.84,-11.86,21.189620,True,,True,True
87,114730.0,390157.0,9,5028_3,5028,3,-0.02,-1.49,-2.49,17,...,17.0,,17,-0.84,-11.86,21.189620,True,,True,True
88,114783.0,391892.0,7,5029_1,5029,1,-0.12,4.00,3.00,13,...,NaN,12,"11, 12, 13",5.57,2.56,53.413481,True,"11, 12",True,True
89,114783.0,391892.0,9,5029_3,5029,3,-0.01,-4.45,-5.45,17,...,17.0,,17,-3.31,-14.85,53.413481,True,,True,True
96,115212.0,394265.0,6,5187_1,5187,1,3.30,3.30,2.30,11,...,11.0,,11,5.51,2.03,40.853396,True,,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4071,121255.0,391641.0,3,B50E0539_1,B50E0539,1,17.87,16.61,16.33,5,...,5.0,,5,17.15,16.01,10.295630,True,,True,True
4072,121880.0,392960.0,4,B50E0541_1,B50E0541,1,16.60,14.80,14.52,7,...,NaN,6,"5, 6, 7",15.79,14.25,31.622777,True,"5, 6",True,True
4073,121563.0,392300.0,5,B50E0548_1,B50E0548,1,16.82,15.05,14.78,9,...,NaN,,"6, 7",15.05,14.60,51.662365,False,"6, 7",True,True
4074,122178.0,392044.0,5,B50E0563_1,B50E0563,1,16.14,15.08,14.83,9,...,NaN,8,"7, 8, 9",15.47,14.38,28.635642,True,"7, 8",True,True


In [34]:
IPF_Mdl_.to_csv('check_OBS_L_values.csv', index=None)

# 6. Stats

## 6.0. KGE of all Vs incorrectly matched. 